In [3]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [18]:
df1 = pd.read_csv('./ethos/hs_data/en_dataset_with_stop_words.csv')
df2 = pd.read_csv('./ethos/hs_data/hate-speech-and-offensive-language.csv')
df3 = pd.read_csv('./ethos/ethos_data/Ethos_Dataset_Binary.csv',sep=';')
df4 = pd.read_csv('./ethos/ethos_data/Ethos_Dataset_Multi_Label.csv',sep=';')

In [19]:
df1

,HITId,tweet,sentiment,directness,annotator_sentiment,target,group
0,0,If America had another 8 years of Obama's ideo...,fearful_abusive_hateful_disrespectful_normal,indirect,anger_fear_shock_sadness_disgust,origin,other
1,1,Most Canadians have never met seen or associat...,offensive,indirect,sadness_indifference,disability,special_needs
2,2,Hahaha grow up faggot @URL,offensive,indirect,shock_disgust,sexual_orientation,women
3,3,@user queue is fucking retarded it makes every...,offensive_hateful,direct,shock_disgust,disability,special_needs
4,4,@user Que ce ne soit pas des Burundais refugie...,hateful_normal,indirect,shock_disgust,origin,other
...,...,...,...,...,...,...,...
5642,5642,@user @user @user Maybe but @user who lied abo...,hateful,indirect,sadness_disgust,origin,refugees
5643,5643,@user @user You\u2019re a retard :) x,hateful,direct,anger_shock_indifference_disgust,other,individual
5644,5644,(2/2) like no you fucking twat I'm doing damag...,hateful,indirect,anger_shock_indifference_disgust,other,special_needs
5645,5645,If your bf is retarded guess what u r 2,normal,indirect,anger_shock_indifference_disgust,disability,women


In [23]:
df1.sentiment.value_counts()

offensive                                          2954
normal                                              661
hateful                                             315
offensive_hateful                                   252
offensive_disrespectful                             145
abusive_offensive                                   115
offensive_normal                                    114
hateful_disrespectful                                96
abusive_hateful                                      90
fearful_abusive_offensive_hateful_disrespectful      85
hateful_normal                                       79
disrespectful_normal                                 69
fearful_abusive_offensive_hateful_normal             62
fearful_offensive_hateful_disrespectful_normal       61
abusive_offensive_hateful_disrespectful_normal       53
fearful_abusive_offensive_disrespectful_normal       53
abusive_disrespectful                                50
fearful_abusive_hateful_disrespectful_normal    

In [20]:
df2

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [21]:
df3

,comment,isHate
0,You should know women's sports are a joke,1.0
1,You look like Sloth with deeper Down’s syndrome,1.0
2,You look like Russian and speak like Indian. B...,1.0
3,"Women deserve to be abused, I guess.",1.0
4,Women are made for making babies and cooking d...,1.0
...,...,...
993,From the midnight sun where the hot springs blow,0.0
994,Don't say I'm not your type,0.0
995,And therefore never send to know for whom the...,0.0
996,And I can't stand another day,0.0


In [22]:
df4

,comment,violence,directed_vs_generalized,gender,race,national_origin,disability,religion,sexual_orientation
0,You should know women's sports are a joke,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,You look like Sloth with deeper Down’s syndrome,0.000000,0.833333,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
2,You look like Russian and speak like Indian. B...,1.000000,0.500000,0.142857,0.142857,0.714286,0.000000,0.000000,0.000000
3,"Women deserve to be abused, I guess.",0.666667,0.166667,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Women are made for making babies and cooking d...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
428,All people are fucked up because of these quee...,0.000000,0.000000,0.571429,0.000000,0.000000,0.000000,0.000000,0.571429
429,Actually THIS WEIRD PEOPLE are HARMFUL! They n...,0.000000,1.000000,0.000000,0.000000,0.333333,0.666667,0.000000,0.000000
430,ABU DA BI ABA DU BA BA HAMUD! See? I’m an arab...,0.000000,0.000000,0.000000,0.285714,0.000000,0.000000,0.857143,0.000000
431,a homo? so you fuck your brother instead of yo...,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
